In [1]:
import pandas as pd

# Load the dataset
df_sampled = pd.read_csv("C:/Users/Guest01/Documents/Manpreet_thesis/Datasets/Glassdoor_job_reviews2/all_reviews.csv")

# Downcast numerical columns to save memory
def downcast(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype == "int64"]
    df[float_cols] = df[float_cols].astype("float32")
    df[int_cols] = df[int_cols].astype("int32")
    return df

df_sampled = downcast(df_sampled)
df_sampled.head()


C:\Users\Guest01\AppData\Local\Temp\ipykernel_19168\4126841228.py:4: DtypeWarning: Columns (5,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sampled = pd.read_csv("C:/Users/Guest01/Documents/Manpreet_thesis/Datasets/Glassdoor_job_reviews2/all_reviews.csv")


,rating,title,status,pros,cons,advice,Recommend,CEO Approval,Business Outlook,Career Opportunities,Compensation and Benefits,Senior Management,Work/Life Balance,Culture & Values,Diversity & Inclusion,firm_link,date,job,index
0,5.0,Good,"Current Employee, more than 10 years",Knowledge gain of complete project,Financial growth and personal growth,NaN,v,o,v,3,3,3,3,3.0,3.0,Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm,"Nov 19, 2022",Manager Design,NaN
1,4.0,Good,"Former Employee, less than 1 year","Good work,good work , flexible, support","Good,work, flexible,good support, good team work",NaN,v,o,o,4,4,4,4,4.0,4.0,Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm,"Jan 29, 2022",Anonymous Employee,NaN
2,4.0,"Supervising the manufacturing the processes, e...","Current Employee, more than 1 year",This company is a best opportunity for me to l...,"Monthly Target work,Maintain production schedu...",NaN,v,o,v,2,3,2,2,2.0,2.0,Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm,"Aug 12, 2021",Production Engineer,NaN
3,1.0,terrible,"Current Employee, more than 1 year",I wish there were some to list,too many to list here,NaN,x,x,x,1.0,3.0,1.0,3.0,1.0,NaN,https://www.glassdoor.com/Reviews/Calgary-Flam...,"Sep 24, 2020",Senior Account Executive,NaN
4,4.0,"It could be so good, but it isn’t","Current Employee, more than 3 years",Fast Paced. Endless challenges. Inclusive envi...,The biggest perk of the job provides no value ...,NaN,o,o,o,3.0,3.0,3.0,1.0,4.0,5.0,https://www.glassdoor.com/Reviews/Calgary-Flam...,"Mar 25, 2023",Assistant Manager,NaN


In [2]:
import pandas as pd
from category_encoders import TargetEncoder

# List of relevant features
relevant_features = [
    'pros', 'cons', 'Recommend', 'Career Opportunities', 'Compensation and Benefits', 
    'Senior Management', 'Work/Life Balance', 'Culture & Values', 'Diversity & Inclusion', 
    'job', 'status'
]

# Fill null values with mode for categorical columns and median for numerical columns
def fill_nulls(df, features):
    for column in features:
        if df[column].dtype == 'object':
            df[column] = df[column].fillna(df[column].mode()[0])
        else:
            df[column] = df[column].fillna(df[column].median())
    return df

# Fill null values
df_sampled = fill_nulls(df_sampled, relevant_features + ['rating'])


In [3]:
# Apply infer_objects() to ensure proper type inference
# df_sampled = df_sampled.infer_objects(copy=False)

# Opt-in to the future behavior for downcasting
pd.set_option('future.no_silent_downcasting', True)

# Apply target encoding to only the relevant categorical columns
encoder = TargetEncoder(cols=relevant_features)
df_sampled[relevant_features] = encoder.fit_transform(df_sampled[relevant_features], df_sampled['rating'])

# Ensure df_sampled contains only the relevant features + 'rating'
df_sampled = df_sampled[relevant_features + ['rating']]

# Downcast again to save memory after target encoding
# df_sampled = downcast(df_sampled)

In [4]:
# Define X and y, ensuring only relevant target-encoded features are in X
X = df_sampled.drop(columns=['rating'])  # X contains only the relevant features after encoding
y = df_sampled['rating']

# Check if any non-numeric columns are still present (shouldn't be after encoding)
non_numeric_columns_in_X = X.select_dtypes(include=['object']).columns
print("Non-numeric columns in X:", non_numeric_columns_in_X)  # Should print an empty index

# Print the shapes of X and y
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


Non-numeric columns in X: Index([], dtype='object')
Shape of X: (9901889, 11)
Shape of y: (9901889,)


In [5]:
from sklearn.model_selection import train_test_split

# Function to sample data
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=None, random_state=42)  # Fix for 1.0 sample size
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
import time
import psutil

# Function to calculate and return metrics for Linear Regression
def calculate_metrics(X_train, X_test, y_train, y_test):
    lr = LinearRegression()

    # Define hyperparameters for RandomizedSearchCV
    param_distributions = {
        'fit_intercept': [True, False]  
    }

    random_search = RandomizedSearchCV(lr, param_distributions, n_iter=2, cv=10, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }


In [7]:
import gc  # Garbage Collector

# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]  

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        # Sample data based on the defined sizes
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        
        # Calculate metrics
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Call garbage collection after each iteration to free up memory
        gc.collect()

        # Accumulate total metrics
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


Metrics for sample size 1.0:
RMSE: 0.5689538534461835
MAPE: 0.158075892649011
R2: 0.7927148415269923
nRMSE: 0.14223846336154589
Execution Time (Raw): 41.04569673538208
Normalized Time (s/MB): 0.05659633162684072
Average CPU Usage: 51.5
Sample Size: 1.0
--------------------------------------------------
Metrics for sample size 0.5:
RMSE: 0.568952147443451
MAPE: 0.15802029367832837
R2: 0.7927160846102915
nRMSE: 0.14223803686086275
Execution Time (Raw): 19.309714317321777
Normalized Time (s/MB): 0.04260067835884766
Average CPU Usage: 55.0
Sample Size: 0.5
--------------------------------------------------
Metrics for sample size 0.25:
RMSE: 0.5689544729355875
MAPE: 0.1580157786574034
R2: 0.792714390133515
nRMSE: 0.14223861823389689
Execution Time (Raw): 9.871439456939697
Normalized Time (s/MB): 0.043556316818906995
Average CPU Usage: 52.05
Sample Size: 0.25
--------------------------------------------------
Metrics for sample size 0.125:
RMSE: 0.5689623488437784
MAPE: 0.15789191118107618


In [8]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 1 minutes and 16.78 seconds
Total Normalized Execution Time for Entire Process: 0.05029434 seconds per MB
Total Average CPU Usage for Entire Process: 45.45%


,RMSE,MAPE,R2,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,0.568954,0.158076,0.792715,0.142238,41.045697,0.056596,51.50,1.000
1,0.568952,0.158020,0.792716,0.142238,19.309714,0.042601,55.00,0.500
2,0.568954,0.158016,0.792714,0.142239,9.871439,0.043556,52.05,0.250
3,0.568962,0.157892,0.792709,0.142241,5.592719,0.049354,51.95,0.125
4,1.581569,0.170052,-0.656530,0.395392,0.053064,7.244987,37.00,100.000
5,0.589389,0.167253,0.787549,0.147347,0.052399,0.715423,49.55,1000.000
6,0.586754,0.158066,0.770349,0.146689,0.091404,0.124797,31.45,10000.000
7,0.563960,0.160216,0.798863,0.140990,0.762900,0.104161,35.10,100000.000


Run samples separately